In [2]:
from brickschema import Graph
from rdflib import Namespace
import os
from string import Template


In [685]:
#NOTE: There is entering chilled water is equivalent to return chilled water, right?
# Hot water entering flow sensor wasn't automatically made a brick point. Should it have been?

#TODO: Investigate assumptions 
# - bldg42 chilled water bypass valve, applies to bldg 35 as well
# - bldg4 Supply_Air_flow 
# - smc, removing reheat coils, VAV is an RVAV
# - bldg26_pump_fix just replaced chilled_water_pump_speed with motor speed sensor. This may not be the best change
# - cooling_pct_fix also not sure of a good change. We should definitely have a change log for these. 
# - bldg44 adding a chiller, because there is a meter measuring ns2:chiller. Should I add any feeds relationships to the AHU?
# - bldg5 has a '/' in some point names, throwing off the namespace. Not sure how to programmatically fix this. 


In [472]:
Graph(load_brick_nightly=True).serialize('Brick_nightly.ttl', format = 'ttl')

<Graph identifier=N0b569217ce244af0946405c055872c9a (<class 'brickschema.graph.Graph'>)>

# Validation Class

In [133]:

class BasicValidationInterface:

    def __init__(self, graph_path: str, name: str):
        self.name = name
        self.graph_path = graph_path
        self.ontology_graph = Graph()
        self.ontology_graph.load_file('Brick_nightly.ttl')
        self.ontology_graph.parse('https://qudt.org/2.1/vocab/quantitykind', format='ttl')
        self.ontology_graph.parse('https://qudt.org/2.1/schema/qudt', format='ttl')
        self.ontology_graph = self.ontology_graph.skolemize()
        
        self.graph = Graph()
        self.graph.load_file(self.graph_path)

        self.full_graph = self.graph + self.ontology_graph

    def validate(self):
        valid, report_graph, report = self.full_graph.validate()
        print(f"Graph <{self.graph_path}> is valid? {valid}")
        if not valid:
            print("-" * 79)
            print(report)
            print("-" * 79)
        return valid, report_graph
    
    def update_graph(self, query):
        self.full_graph.update(query)
    
    def save_clean_graph(self, filename):
        self.clean_graph = self.full_graph - self.ontology_graph
        self.fix_clean_namespaces()
        self.clean_graph.serialize(filename, format = 'ttl')

    def fix_clean_namespaces(self):
        self.clean_graph.bind(self.name, 'http://buildsys.org/ontologies/' + self.name + '#')
        #TODO: Fix when a '/' is in the point name (may be easier to just do this manually ...)

# Area correction query pattern

In [109]:
area_query_format = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this brick:value "$old" }
INSERT {?this brick:value $new }
WHERE {
?this brick:value "$old"
}
"""


def format_area_query(report_graph):
    res = report_graph.query(""" 
        select ?replace
        WHERE {
        ?s a sh:ValidationResult ;
            sh:value ?value .
        ?val brick:value ?replace .
        }
        """)
    if len(res) == 1:
        print('1 Invalid Shape')
        old = iter(res).__next__().get('replace')
        new = int(old.rsplit('^^')[0])
    else:
        raise ValueError('multiple errors')
    return Template(area_query_format).substitute(old=old, new=new)


# Other Queries

In [110]:
bldg43_add_chiller_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg43>

INSERT DATA {<http://buildsys.org/ontologies/bldg43#chiller> a brick:Chiller }
"""


In [111]:
lev_p_sens_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Supply_Pressure_Sensor }
INSERT {?this a brick:Pressure_Sensor ;
            brick:hasSubstance brick:Leaving_Chilled_Water }
WHERE {
?this a brick:Supply_Pressure_Sensor   .
}
"""


In [112]:
ent_p_sens_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Return_Pressure_Sensor }
INSERT {?this a brick:Pressure_Sensor ;
            brick:hasSubstance brick:Entering_Chilled_Water }
WHERE {
?this a brick:Return_Pressure_Sensor   .
}
"""


In [113]:
# This may cause mistakes
heating_demand_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Heating_Demand    }
INSERT {?this a brick:Heating_Demand_Sensor }
WHERE {
?this a brick:Heating_Demand   .
}
"""


In [114]:
# This may cause mistakes
cooling_demand_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Cooling_Demand   }
INSERT {?this a brick:Cooling_Demand_Sensor }
WHERE {
?this a brick:Cooling_Demand   .
}
"""


In [115]:
# This may cause mistakes
cooling_pct_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Cooling_Request_Percent_Setpoint  }
INSERT {?this a brick:Leaving_Chilled_Water_Temperature_Setpoint }
WHERE {
?this a brick:Cooling_Request_Percent_Setpoint  .
}
"""


In [116]:
# This may cause mistakes
bldg26_pump_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Chilled_Water_Pump_VFD_Speed }
INSERT {?this a brick:Motor_Speed_Sensor .}
WHERE {
?this a brick:Chilled_Water_Pump_VFD_Speed .
}
"""


In [117]:
boiler_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Hot_Water_Supply_Flow_Sensor}
INSERT {?this a brick:Hot_Water_Entering_Flow_Sensor ;
    a brick:Point . }
WHERE {
?this a brick:Hot_Water_Supply_Flow_Sensor .
}
"""


In [118]:
r_coil_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE { ?this a brick:Reheat_Coil .
        ?vav brick:hasPart ?this .}
INSERT { ?vav a brick:RVAV }
WHERE {
    ?this a brick:Reheat_Coil .
    ?vav brick:hasPart ?this .
    }
"""

#     Filter (?s = ?this) 
#     BIND (?this as ?s)
#     }
# UNION
#     {
#     ?this a brick:Reheat_Coil .
#     ?vav brick:hasPart ?this .
#     Filter (?o = ?this) 
#     BIND (?this as ?o)
#     }
# }
# """


In [119]:
chwr_ts_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Min_Chilled_Water_Supply_Temperature_Setpoint}
INSERT {?this a brick:Min_Water_Temperature_Setpoint ; 
            brick:ofSubstance brick:Leaving_Chilled_Water }
WHERE {
?this a brick:Min_Chilled_Water_Supply_Temperature_Setpoint
}
"""


In [120]:
# ASSUMING EVERYTHING WITH RM, WITH NO DECLARED TYPE, IS A ZONE
# FOR BLDG6 SPECIFICALLY
bldg6_rm_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {}
INSERT {?this a brick:HVAC_Zone }
WHERE {
    ?this brick:hasPoint ?that .
    FILTER REGEX (str(?this), "VAVRM") .
    FILTER NOT EXISTS {
        ?this a ?y .
  }
}
"""


In [121]:
chwsp_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Chilled_Water_Supply_Pressure}
INSERT {?this a brick:Pressure_Sensor ;
            brick:hasSubstance brick:Leaving_Chilled_Water}
WHERE {
    ?this a brick:Chilled_Water_Supply_Pressure
}
"""
chwrt_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Chilled_Water_Return_Pressure}
INSERT {?this a brick:Pressure_Sensor ;
            brick:hasSubstance brick:Entering_Chilled_Water}
WHERE {
    ?this a brick:Chilled_Water_Return_Pressure
}
"""


In [122]:
#bldg42: ASSUMING chilled water bypass valve refers to a bypass valve status
#investigate this further
bldg42_chwbp_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg42>

DELETE {?this a brick:Chilled_Water_Bypass_Valve .
        ?chiller brick:hasPoint ?this .}
INSERT {?this a brick:Bypass_Valve ;
            brick:hasSubstance brick:Chilled_Water;
            brick:hasPoint bldg:BypassValveStatus .
        bldg:BypassValveStatus a brick:Valve_Status .
        ?chiller brick:hasPart ?this .}
WHERE {
    ?this a brick:Chilled_Water_Bypass_Valve.
    ?chiller brick:hasPoint ?this .
}
"""


In [123]:
oad_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Outside_Air_Damper_Command}
INSERT {?this a brick:Damper_Command ;
            brick:hasSubstance brick:Outside_Air}
WHERE {
    ?this a brick:Outside_Air_Damper_Command
}
"""


In [124]:
# ASSUMING MODE MEANS MODE STATUS
mode_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Mode}
INSERT {?this a brick:Mode_Status}
WHERE {
    ?this a brick:Mode
}
"""


In [125]:
deprecation_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a ?class}
INSERT {?this a ?newClass}
WHERE {
?this a ?class .
?class owl:deprecated true .
?class brick:deprecatedInVersion ?depver .
?class brick:isReplacedBy ?newClass .
}
"""


In [126]:
#
ft2_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this brick:hasUnits unit:FT_2}
INSERT {?this brick:hasUnit unit:FT2}
WHERE {
?this brick:hasUnits unit:FT_2
}
"""


In [127]:
chwv_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Chilled_Water_Valve_Command }
INSERT {?this a brick:Valve_Command ;
                brick:hasSubstance brick:Chilled_Water }
WHERE {
?this a brick:Chilled_Water_Valve_Command
}
"""


In [128]:
hwv_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Hot_Water_Valve_Command }
INSERT {?this a brick:Valve_Command ;
                brick:hasSubstance brick:Hot_Water }
WHERE {
?this a brick:Hot_Water_Valve_Command
}
"""


In [129]:
# ASSUMING that flow refers to a Sensor
bldg4_supflow_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Supply_Air_flow }
INSERT {?this a brick:Supply_Air_Flow_Sensor}
WHERE {
?this a brick:Supply_Air_flow
}
"""


In [40]:
chws_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Chilled_Water_Supply_Temperature_Setpoint}
INSERT {?this a brick:Supply_Chilled_Water_Temperature_Setpoint }
WHERE {
?this a brick:Chilled_Water_Supply_Temperature_Setpoint
}
"""


In [41]:
chwr_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Chilled_Water_Return_Temperature_Setpoint}
INSERT {?this a brick:Return_Chilled_Water_Temperature_Setpoint }
WHERE {
?this a brick:Chilled_Water_Return_Temperature_Setpoint
}
"""


In [42]:
meters_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?meter brick:isPointOf ?this}
INSERT {?meter brick:meters ?this }
WHERE {
?meter a brick:Electrical_Meter ;
    brick:isPointOf ?this .
}
"""


In [43]:
electricalmeter_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?meter a brick:Electric_Meter }
INSERT {?meter a brick:Electrical_Meter  }
WHERE {
?meter a brick:Electric_Meter 
}
"""


# Correcting All the Buildings

In [104]:
# Deprecation query should come last
other_query_list = [heating_demand_fix, bldg42_chwbp_fix, ent_p_sens_fix, lev_p_sens_fix,
                    chwr_ts_fix, r_coil_fix, boiler_fix, bldg26_pump_fix, cooling_pct_fix, cooling_demand_fix,
                    chwsp_fix, chwrt_fix, oad_fix, mode_fix, chwv_fix, ft2_fix, bldg4_supflow_fix,
                    hwv_fix, chws_fix, chwr_fix, meters_fix, electricalmeter_fix, meters_fix, deprecation_fix]


In [132]:
# last_fail = 11
# last_fail = i
i = 0
for file in os.listdir('graphs'):
    print(file)
    # if i < last_fail:
    #     i += 1
    #     continue
    # file = 'bldg2.ttl'
    path = os.path.join('graphs', file)
    bv = BasicValidationInterface(graph_path=path, name = file.split('.')[0])
    for q in other_query_list:
        bv.update_graph(q)
    if file == 'bldg6.ttl':
        bv.update_graph(bldg6_rm_fix)
    if file == 'bldg43.ttl':
        bv.update_graph(bldg43_add_chiller_fix)
    valid, report_graph = bv.validate()
    if valid:
        print("Graph is valid")
        bv.save_clean_graph(os.path.join('graphs_clean', file))
        continue
    area_query = format_area_query(report_graph)
    bv.update_graph(area_query)
    valid, final_report = bv.validate()
    if not valid:
        raise ValueError('Not validated')
    bv.save_clean_graph(os.path.join('graphs_clean', file))
    i += 1


bldg5.ttl
Graph <graphs/bldg5.ttl> is valid? False
-------------------------------------------------------------------------------
Validation Report
Conforms: False
Results (1):
Constraint Violation in NodeConstraintComponent (http://www.w3.org/ns/shacl#NodeConstraintComponent):
	Severity: sh:Violation
	Source Shape: bsh:hasareaShape
	Focus Node: ns2:bldg5
	Value Node: [ brick:hasUnit unit:FT2 ; brick:value Literal("99922^^<http://www.w3.org/2001/XMLSchema#integer>") ]
	Result Path: brick:area
	Message: Value does not conform to Shape bsh:AreaShape

-------------------------------------------------------------------------------
1 Invalid Shape
Graph <graphs/bldg5.ttl> is valid? True
bldg4.ttl
Graph <graphs/bldg4.ttl> is valid? False
-------------------------------------------------------------------------------
Validation Report
Conforms: False
Results (1):
Constraint Violation in NodeConstraintComponent (http://www.w3.org/ns/shacl#NodeConstraintComponent):
	Severity: sh:Violation
	Sou

# Working it out

In [286]:
deprecation_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a ?class}
INSERT {?this a ?newClass}
WHERE {
?this a ?class .
?class owl:deprecated true .
?class brick:deprecatedInVersion ?depver .
?class brick:isReplacedBy ?newClass .
}
"""


In [287]:
#
ft2_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this brick:hasUnits unit:FT_2}
INSERT {?this brick:hasUnit unit:FT2}
WHERE {
?this brick:hasUnits unit:FT_2
}
"""


In [288]:
bldg1_area_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this brick:value "9973^^<http://www.w3.org/2001/XMLSchema#integer>" }
INSERT {?this brick:value 9973 }
WHERE {
?this brick:value "9973^^<http://www.w3.org/2001/XMLSchema#integer>"
}
"""


In [289]:
# bldg 1


In [370]:
graph = 'graphs/bldg1.ttl'
bv = BasicValidationInterface(graph_path=graph, nightly=True)
bv.graph.update(deprecation_fix)
# bv.graph.update(bldg1_area_fix)
bv.graph.update(ft2_fix)
# Loading qudt fixes QUDT related validation errors
bv.graph.parse('https://qudt.org/2.1/schema/qudt', format='ttl')
bv.graph.parse('https://qudt.org/2.1/vocab/quantitykind', format='ttl')
report_graph = bv.validate()


Graph <graphs/bldg1.ttl> is valid? False
-------------------------------------------------------------------------------
Validation Report
Conforms: False
Results (1):
Constraint Violation in NodeConstraintComponent (http://www.w3.org/ns/shacl#NodeConstraintComponent):
	Severity: sh:Violation
	Source Shape: bsh:hasareaShape
	Focus Node: ns2:bldg1
	Value Node: [ brick:hasUnit unit:FT2 ; brick:value Literal("9973^^<http://www.w3.org/2001/XMLSchema#integer>") ]
	Result Path: brick:area
	Message: Value does not conform to Shape bsh:AreaShape

-------------------------------------------------------------------------------


In [388]:
new = format_area_query(report_graph)
print(new)


1 Invalid Shape

PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this brick:value "9973^^<http://www.w3.org/2001/XMLSchema#integer>" }
INSERT {?this brick:value 9973 }
WHERE {
?this brick:value "9973^^<http://www.w3.org/2001/XMLSchema#integer>"
}



In [389]:
bv.graph.update(new)


In [390]:
bv.validate()


Graph <graphs/bldg1.ttl> is valid? True


<Graph identifier=N8cc758c050614bad8073c97b06b3a6bc (<class 'rdflib.graph.Graph'>)>

In [348]:
if len(res) == 1:
    print('1 Invalid Shape')
    old = iter(res).__next__().get('replace')
    new = int(old.rsplit('^^')[0])


1 Invalid Shape


In [ ]:
Template(area_query_format).substitute(old='a', new='b')


In [298]:
bv.graph.serialize('graphs_clean/bldg1.ttl')


<Graph identifier=N596ed28253704a03bb8e96a39a54213f (<class 'brickschema.graph.Graph'>)>

In [140]:
# bldg2


In [141]:
chwv_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Chilled_Water_Valve_Command }
INSERT {?this a brick:Valve_Command ;
                brick:hasSubstance brick:Chilled_Water }
WHERE {
?this a brick:Chilled_Water_Valve_Command
}
"""


In [142]:
hwv_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Hot_Water_Valve_Command }
INSERT {?this a brick:Valve_Command ;
                brick:hasSubstance brick:Hot_Water }
WHERE {
?this a brick:Hot_Water_Valve_Command
}
"""


In [143]:
chws_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Chilled_Water_Supply_Temperature_Setpoint}
INSERT {?this a brick:Supply_Chilled_Water_Temperature_Setpoint }
WHERE {
?this a brick:Chilled_Water_Supply_Temperature_Setpoint
}
"""


In [144]:
chwr_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Chilled_Water_Return_Temperature_Setpoint}
INSERT {?this a brick:Return_Chilled_Water_Temperature_Setpoint }
WHERE {
?this a brick:Chilled_Water_Return_Temperature_Setpoint
}
"""


In [145]:
meters_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?meter brick:isPointOf ?this}
INSERT {?meter brick:meters ?this }
WHERE {
?meter a brick:Electric_Meter ;
    brick:isPointOf ?this .
}
"""


In [146]:
electricalmeter_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?meter a brick:Electric_Meter }
INSERT {?meter a brick:Electrical_Meter  }
WHERE {
?meter a brick:Electric_Meter 
}
"""


In [233]:
# couldn't figure out how to generalize this (it is probably possible, but I couldn't crack it)
bldg2_area_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this brick:value "26793^^<http://www.w3.org/2001/XMLSchema#integer>" }
INSERT {?this brick:value 26793 }
WHERE {
?this brick:value "26793^^<http://www.w3.org/2001/XMLSchema#integer>"
}
"""


In [234]:
graph = 'graphs/bldg2.ttl'
bv = BasicValidationInterface(graph_path=graph, nightly=True)
bv.graph.update(bldg2_area_fix)
bv.graph.update(chwv_fix)
bv.graph.update(ft2_fix)
bv.graph.update(bldg2_area_fix)
bv.graph.update(hwv_fix)
# bv.graph.update(bldg2_area_fix)
# Loading qudt fixes QUDT related validation errors
bv.graph.parse('https://qudt.org/2.1/schema/qudt', format='ttl')
bv.graph.parse('https://qudt.org/2.1/vocab/quantitykind', format='ttl')

bv.graph.update(chws_fix)
bv.graph.update(chwr_fix)
bv.graph.update(meters_fix)
bv.graph.update(electricalmeter_fix)
bv.graph.update(deprecation_fix)
bv.validate()


Graph <graphs/bldg2.ttl> is valid? True


In [ ]:
bv.graph.serialize('graphs_clean/bldg2.ttl')


In [230]:
# bldg3


In [238]:
# couldn't figure out how to generalize this (it is probably possible, but I couldn't crack it)
bldg3_area_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this brick:value "42946" }
INSERT {?this brick:value 42946 }
WHERE {
?this brick:value "42946"
}
"""


In [239]:
graph = 'graphs/bldg3.ttl'
bv = BasicValidationInterface(graph_path=graph, nightly=True)
bv.graph.update(bldg1_area_fix)
bv.graph.update(chwv_fix)
bv.graph.update(ft2_fix)
bv.graph.update(bldg3_area_fix)
bv.graph.update(hwv_fix)
bv.graph.update(bldg2_area_fix)
# Loading qudt fixes QUDT related validation errors
bv.graph.parse('https://qudt.org/2.1/schema/qudt', format='ttl')
bv.graph.parse('https://qudt.org/2.1/vocab/quantitykind', format='ttl')

bv.graph.update(chws_fix)
bv.graph.update(chwr_fix)
bv.graph.update(meters_fix)
bv.graph.update(electricalmeter_fix)
bv.graph.update(deprecation_fix)
bv.validate()


Graph <graphs/bldg3.ttl> is valid? True


In [ ]:
bv.graph.serialize('graphs_clean/bldg3.ttl')


In [240]:
# bldg4


In [251]:
# couldn't figure out how to generalize this (it is probably possible, but I couldn't crack it)
bldg4_area_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this brick:value "32792^^<http://www.w3.org/2001/XMLSchema#integer>" }
INSERT {?this brick:value 32792 }
WHERE {
?this brick:value "32792^^<http://www.w3.org/2001/XMLSchema#integer>"
}
"""


In [252]:
# ASSUMING that flow refers to a Sensor
bldg4_supflow_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Supply_Air_flow }
INSERT {?this a brick:Supply_Air_Flow_Sensor}
WHERE {
?this a brick:Supply_Air_flow
}
"""


In [253]:
graph = 'graphs/bldg4.ttl'
bv = BasicValidationInterface(graph_path=graph, nightly=True)
bv.graph.update(bldg1_area_fix)
bv.graph.update(chwv_fix)
bv.graph.update(ft2_fix)
bv.graph.update(bldg4_supflow_fix)
bv.graph.update(bldg4_area_fix)
bv.graph.update(hwv_fix)
# Loading qudt fixes QUDT related validation errors
bv.graph.parse('https://qudt.org/2.1/schema/qudt', format='ttl')
bv.graph.parse('https://qudt.org/2.1/vocab/quantitykind', format='ttl')

bv.graph.update(chws_fix)
bv.graph.update(chwr_fix)
bv.graph.update(meters_fix)
bv.graph.update(electricalmeter_fix)
bv.graph.update(deprecation_fix)
bv.validate()


Graph <graphs/bldg4.ttl> is valid? True


In [ ]:
bv.graph.serialize('graphs_clean/bldg4.ttl')


In [254]:
# bldg5


In [257]:
# couldn't figure out how to generalize this (it is probably possible, but I couldn't crack it)
bldg5_area_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this brick:value "99922^^<http://www.w3.org/2001/XMLSchema#integer>" }
INSERT {?this brick:value 99922}
WHERE {
?this brick:value "99922^^<http://www.w3.org/2001/XMLSchema#integer>"
}
"""


In [258]:
graph = 'graphs/bldg5.ttl'
bv = BasicValidationInterface(graph_path=graph, nightly=True)
bv.graph.update(bldg1_area_fix)
bv.graph.update(chwv_fix)
bv.graph.update(ft2_fix)
bv.graph.update(bldg5_area_fix)
bv.graph.update(hwv_fix)
# Loading qudt fixes QUDT related validation errors
bv.graph.parse('https://qudt.org/2.1/schema/qudt', format='ttl')
bv.graph.parse('https://qudt.org/2.1/vocab/quantitykind', format='ttl')

bv.graph.update(chws_fix)
bv.graph.update(chwr_fix)
bv.graph.update(meters_fix)
bv.graph.update(electricalmeter_fix)
bv.graph.update(deprecation_fix)
bv.validate()


Graph <graphs/bldg5.ttl> is valid? True


In [260]:
bv.graph.serialize('graphs_clean/bldg5.ttl')


<Graph identifier=Nb9469e785c5e45fe8374b40bdde783d2 (<class 'brickschema.graph.Graph'>)>

In [ ]:
# Running all for rest of process


In [262]:
area_query_format = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this $old  }
INSERT {?this brick:value $new }
WHERE {
?this brick:value %old 
}
"""


In [265]:
Template(area_query_format).substitute(old='a', new='b')



PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a  }
INSERT {?this brick:value b }
WHERE {
?this brick:value %old 
}

